visual encoder -> 196 token 768
llm -> tokenizer -> embed -> 10 token 4096
connector = project (768 -> 4096) + concat
project -> 196 token 4096
206 token 4096
196 token + 10 token
position_ids
llm(position_ids=, inputs_embed=, mask=[1, 1, 1, 1, 1, 1])
decode

<image>


In [1]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(
    "/pasteur/u/yiming/small-vlm/outputs/2025-05-12/01-42-28/checkpoint-3", trust_remote_code=True
)


/pasteur/u/yiming/small-vlm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: The checkpoint you are trying to load has model type `vlm` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [1]:
import torch

image = torch.load("/pasteur/u/yiming/small-vlm/outputs/2025-05-11/21-06-37/images.pt")


In [ ]:
image


tensor([[[[ 0.4707,  0.4551,  0.4707,  ...,  0.4707,  0.5586,  0.5156],
          [ 0.5586,  0.5742,  0.5430,  ..., -0.7695, -0.5820, -0.2734],
          [ 0.6328,  0.6016,  0.5859,  ..., -1.5312, -1.5625, -1.5312],
          ...,
          [-0.0845, -0.1279, -0.1572,  ..., -1.3125, -1.2656, -1.2109],
          [-0.1279, -0.1279, -0.1865,  ..., -1.3125, -1.2109, -1.1953],
          [-0.1572, -0.0986, -0.2012,  ..., -1.3125, -1.2109, -1.2344]],

         [[ 0.6484,  0.6328,  0.6484,  ...,  0.7539,  0.8438,  0.8008],
          [ 0.7383,  0.7539,  0.7227,  ..., -0.5195, -0.3262, -0.0413],
          [ 0.8125,  0.7852,  0.7695,  ..., -1.3281, -1.3594, -1.3438],
          ...,
          [ 0.0038, -0.0413, -0.0413,  ..., -1.1094, -1.0625, -0.9883],
          [-0.0413, -0.0413, -0.0713,  ..., -1.1406, -1.0312, -1.0000],
          [-0.0713, -0.0112, -0.0864,  ..., -1.1406, -1.0312, -1.0469]],

         [[ 0.8789,  0.8672,  0.8516,  ...,  0.8672,  0.9531,  0.9102],
          [ 0.9648,  0.9805,  

In [ ]:
from transformers import CLIPVisionModel

model = CLIPVisionModel.from_pretrained(
    "openai/clip-vit-large-patch14-336",
    trust_remote_code=True,
)


/pasteur/u/yiming/small-vlm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-11 21:09:54,385] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_open'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_unlink'
collect2: error: ld returned 1 exit status


In [6]:
model.cuda()
output = model(image)


In [7]:
output


BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.1939, -0.4720, -0.2071,  ...,  0.2971,  0.5958, -0.2120],
         [ 0.3681,  1.0474, -0.3662,  ...,  0.5626, -0.5293, -0.3300],
         [ 1.8667,  0.9950, -0.3714,  ...,  2.3189,  0.1522,  0.9149],
         ...,
         [ 0.4588,  0.8863, -0.3187,  ...,  0.6389,  0.0093, -0.0024],
         [ 0.4796,  0.7912, -0.6391,  ...,  0.4792, -0.0388,  0.1317],
         [ 0.0817,  1.1452, -0.5089,  ...,  0.4315, -0.1363,  0.0750]]],
       device='cuda:0', grad_fn=<AddBackward0>), pooler_output=tensor([[ 0.3666, -0.8041, -0.4335,  ...,  0.7717,  1.3503, -0.4265]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [2]:
from transformers import (
    AutoConfig,
)
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING


def create_dynamic_vlm_class(
    base_language_model_name_or_path: str,  # e.g., "google/gemma-3-4b-it"
):
    """
    Dynamically creates a VLM class that inherits from the specified base LLM's
    *ForCausalLM model class.
    """
    # Determine the base LLM's actual config class and model class
    base_language_model_actual_config = AutoConfig.from_pretrained(
        base_language_model_name_or_path, trust_remote_code=True
    )
    base_language_model_config_class = base_language_model_actual_config.__class__
    if base_language_model_config_class not in MODEL_FOR_CAUSAL_LM_MAPPING:
        raise ValueError(
            f"Config class {base_language_model_config_class.__name__} for LLM {base_language_model_name_or_path} "
            f"not in MODEL_FOR_CAUSAL_LM_MAPPING. Cannot determine parent CausalLM class."
        )
    ParentLLMClass = MODEL_FOR_CAUSAL_LM_MAPPING[base_language_model_config_class]
    return ParentLLMClass


/pasteur/u/yiming/small-vlm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
VLM = create_dynamic_vlm_class("lmsys/vicuna-7b-v1.5")


[2025-05-11 03:20:40,988] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_open'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_unlink'
collect2: error: ld returned 1 exit status


In [7]:
text_config = "lmsys/vicuna-7b-v1.5"
config = AutoConfig.from_pretrained(text_config)


In [1]:
from vlm.models.configuration_vlm import VLMConfig
from transformers import CLIPVisionConfig, AutoConfig

vision_config = {
    "name": "clip",
    "hf_name": "openai/clip-vit-base-patch16",
    "model_type": "huggingface",
}
text_config = "lmsys/vicuna-7b-v1.5"
config = AutoConfig.from_pretrained(text_config)
print(config)
projector_config = {"name": "projector"}


configuration = VLMConfig(config.to_dict(), vision_config, projector_config)


/pasteur/u/yiming/small-vlm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-11 06:30:49,248] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_open'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_unlink'
collect2: error: ld returned 1 exit status


LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 32000
}



In [3]:
print(configuration.vision_config)


VisionConfig {
  "hf_name": "openai/clip-vit-base-patch16",
  "model_type": "vision_model",
  "name": "clip",
  "transformers_version": "4.51.3"
}



In [ ]:
from omegaconf import OmegaConf

a = OmegaConf.create(configuration.vision_config.to_dict())


In [ ]:
a


{'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': None, 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': True, 'chunk_size_feed_forward': 0, 'is_encoder_decoder': False, 'is_decoder': False, 'cross_attention_hidden_size': None, 'add_cross_attention': False, 'tie_encoder_decoder': False, 'max_length': 20, 'min_length': 0, 'do_sample': False, 'early_stopping': False, 'num_beams': 1, 'num_beam_groups': 1, 'diversity_penalty': 0.0, 'temperature': 1.0, 'top_k': 50, 'top_p': 1.0, 'typical_p': 1.0, 'repetition_penalty': 1.0, 'length_penalty': 1.0, 'no_repeat_ngram_size': 0, 'encoder_no_repeat_ngram_size': 0, 'bad_words_ids': None, 'num_return_sequences': 1, 'output_scores': False, 'return_dict_in_generate': False, 'forced_bos_token_id': None, 'forced_eos_token_id': None, 'remove_invalid_values': False, 'exponential_decay_length_penalty': None, 'suppress_tokens': None, 'begin_suppress_

In [4]:
from vlm.models.modeling_vlm import create_dynamic_vlm_class

VLM = create_dynamic_vlm_class("lmsys/vicuna-7b-v1.5", "VLM")


In [5]:
import torch

model = VLM.from_pretrained(
    "lmsys/vicuna-7b-v1.5", config=configuration, torch_dtype=torch.bfloat16
)


Fetching 2 files: 100%|██████████| 2/2 [00:00<00:00, 18600.02it/s]


LanguageModelConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "language_model",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 32000
}

VisionConfig {
  "hf_name": "openai/clip-vit-base-patch16",
  "model_type": "vision_model",
  "name": "clip",
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.3"
}

ProjectorConfig {
  "model_type": "projector",
  "name

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.05s/it]


In [1]:
import torch

inputs_embeds = torch.load("inputs_embeds.pt").to("cuda")
attention_mask = torch.load("attention_mask.pt").to("cuda")


In [2]:
inputs_embeds.shape


torch.Size([1, 623, 4096])

In [1]:
from transformers import AutoModelForCausalLM
import torch

original_model = (
    AutoModelForCausalLM.from_pretrained(
        "lmsys/vicuna-7b-v1.5",
        low_cpu_mem_usage=True,
        trust_remote_code=True,
        # torch_dtype=torch.bfloat16,
    )
    .cuda()
    .eval()
)


/pasteur/u/yiming/small-vlm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-11 03:29:11,636] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_open'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_unlink'
collect2: error: ld returned 1 exit status
Loading checkpoint shards:  50%|█████     | 1/2 [00:07<00:07,  7.38s/it]

: 

In [4]:
original_model.config


LlamaConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 32000
}

In [6]:
output = original_model.generate(attention_mask=attention_mask, inputs_embeds=inputs_embeds)


In [ ]:
output = original_model(attention_mask=attention_mask, inputs_embeds=inputs_embeds)


In [7]:
output


tensor([[  450,  1967,  5692,   304,   367,   263,  3802, 29899,   786, 10322,
           310,   263,  2022, 29915, 29879,  3700, 29892, 10075,  4586,   411,
           263,  3802, 29899,   786,   301,   575,   470,   263, 11758,   301,
           575, 29889,   450,  2022, 29915, 29879,  5076,   526,  1722,   322,
          1009,  2258,  1455,  5680,   526,  7962, 29892,  3704,  1009,   321,
         29891,   295,  1161,   267,   322,   321, 29891,   774,  5727, 29889,
           450,  3239,   310,   278,  1967,   338,  1999,   332,   719,   322,
           714,   310,  8569, 29892,   607, 14661,   393,   278,  1967,   471,
          4586,   411,   263,  1407,  4091,   340, 10809,   310,  1746, 29889,
           450,  2022, 29915, 29879, 19309, 16225,  5692,   304,   367, 10029,
         14294,   322,   756,   263,  7248,   330,   677, 29892,   607,  1033,
           367,  2861,   304,   278,   671,   310,   263, 11013,   470,   263,
          4964,  3578,  2752, 29889,  6811,   497, 2

In [1]:
from vlm.inference.eval import eval_model

eval_model(
    "/pasteur/u/yiming/small-vlm/outputs/2025-05-12/05-41-15/checkpoint-5",
    "Describe this image.",
    "/pasteur/u/yiming/small-vlm/example/image.jpg",
    attn_implementation="eager",
)


/pasteur/u/yiming/small-vlm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-12 06:02:34,215] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_open'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_unlink'
collect2: error: ld returned 1 exit status
Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 15.24it/s]


input_ids tensor([[    1,   319, 13563,  1546,   263, 12758,  5199,   322,   385, 23116,
         21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
           322,  1248,   568,  6089,   304,   278,  5199, 29915, 29879,  5155,
         29889,  3148,  1001, 29901, 29871,  -200, 29871,    13,  4002, 29581,
           445,  1967, 29889,   319,  1799,  9047, 13566, 29901]],
       device='cuda:0')
images_tensor tensor([[[[-0.0112, -0.0112, -0.0112,  ..., -0.0112, -0.0112, -0.0112],
          [-0.0112, -0.0112, -0.0112,  ..., -0.0112, -0.0112, -0.0112],
          [-0.0112, -0.0112, -0.0112,  ..., -0.0112, -0.0112, -0.0112],
          ...,
          [-0.0112, -0.0112, -0.0112,  ..., -0.0112, -0.0112, -0.0112],
          [-0.0112, -0.0112, -0.0112,  ..., -0.0112, -0.0112, -0.0112],
          [-0.0112, -0.0112, -0.0112,  ..., -0.0112, -0.0112, -0.0112]],

         [[-0.0112, -0.0112, -0.0112,  ..., -0.0112, -0.0112, -0.0112],
          [-0.0112, -0.0112, -0.0112,  ..., -0.0

/pasteur/u/yiming/small-vlm/.venv/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


output_ids tensor([[  263, 29871, 29896, 29900, 29889, 29945, 29889,   263, 29871, 29896,
         29900, 29889, 29945, 29889,   263, 29871, 29896, 29900, 29889, 29945,
         29889,   263, 29871, 29896, 29900, 29889, 29945, 29889,   263, 29871,
         29896, 29900, 29889, 29945, 29889,   263, 29871, 29896, 29900, 29889,
         29945, 29889,   263, 29871, 29896, 29900, 29889, 29945, 29889,   263,
         29871, 29896, 29900, 29889, 29945, 29889,   263, 29871, 29896, 29900,
         29889, 29945, 29889,   263, 29871, 29896, 29900, 29889, 29945, 29889,
           263, 29871, 29896, 29900, 29889, 29945, 29889,   263, 29871, 29896,
         29900, 29889, 29945, 29889,   263, 29871, 29896, 29900, 29889, 29945,
         29889,   263, 29871, 29896, 29900, 29889, 29945, 29889,   263, 29871,
         29896, 29900, 29889, 29945, 29889,   263, 29871, 29896, 29900, 29889,
         29945, 29889,   263, 29871, 29896, 29900, 29889, 29945, 29889,   263,
         29871, 29896, 29900, 29889, 2994

In [3]:
model._language_model._language_model.config


LlamaConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 32000
}

In [6]:
inputs_embeds


tensor([[[ 0.0027, -0.0041,  0.0021,  ..., -0.0098,  0.0035, -0.0041],
         [-0.0098, -0.0132, -0.0128,  ...,  0.0089,  0.0129, -0.0085],
         [ 0.0217, -0.0081,  0.0075,  ...,  0.0138, -0.0018, -0.0225],
         ...,
         [-0.0178, -0.0002,  0.0173,  ...,  0.0247,  0.0054,  0.0096],
         [ 0.0063, -0.0166,  0.0113,  ..., -0.0105,  0.0145,  0.0071],
         [ 0.0032,  0.0017,  0.0056,  ..., -0.0054,  0.0162,  0.0022]]],
       device='cuda:0', dtype=torch.bfloat16)

In [4]:
out = model._language_model._language_model.generate(
    attention_mask=attention_mask, inputs_embeds=inputs_embeds
)


In [6]:
out = model._language_model._language_model(
    attention_mask=attention_mask, inputs_embeds=inputs_embeds
)


In [7]:
out


CausalLMOutputWithPast(loss=None, logits=tensor([[[  0.8750,   0.2539,  -0.7070,  ...,   1.7031,   2.7031,   1.2266],
         [-10.1250,  -7.2812,  -1.1875,  ...,  -7.5625,  -9.0000,  -8.9375],
         [ -7.5312,  -7.7500,   8.0000,  ...,  -6.1562,  -4.6562,  -5.7188],
         ...,
         [-12.0000,  -4.4688,   8.0000,  ...,  -5.8750,  -8.7500,  -8.1875],
         [-11.8125,  -4.2500,   6.4375,  ...,  -7.0625,  -7.4688,  -9.3125],
         [ -9.4375,  -4.6875,   9.5625,  ...,  -4.8125,  -5.0312,  -5.8125]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<UnsafeViewBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x7f84105ad090>, hidden_states=None, attentions=None)

In [2]:
from PIL import Image

image = Image.open("/pasteur/u/yiming/small-vlm/example/image.jpg")
image = image.convert("RGB")
model.visual_encoder.preprocessor(image)["pixel_values"]


[tensor([[[-0.1280, -0.1134, -0.1572,  ..., -0.1426,  0.7187,  1.3318],
          [-0.1718, -0.1280, -0.1572,  ...,  0.3099,  0.9960,  1.2296],
          [-0.1572, -0.0988, -0.1134,  ...,  0.7479,  1.2150,  1.0836],
          ...,
          [ 0.0033, -0.0550, -0.0696,  ...,  0.0325,  0.1493,  0.2223],
          [ 0.0179, -0.0550, -0.0696,  ...,  0.1201,  0.2515,  0.3099],
          [ 0.0325, -0.0113, -0.0259,  ...,  0.1785,  0.2223,  0.1639]],
 
         [[-0.0562, -0.0412, -0.0862,  ..., -0.1913,  0.6942,  1.3845],
          [-0.1012, -0.0562, -0.0862,  ...,  0.2589,  0.9793,  1.2795],
          [-0.0862, -0.0262, -0.0412,  ...,  0.7092,  1.2044,  1.1144],
          ...,
          [ 0.0338, -0.0262, -0.0412,  ..., -0.0112,  0.0789,  0.1539],
          [ 0.0488, -0.0262, -0.0412,  ...,  0.0789,  0.1839,  0.2439],
          [ 0.0638,  0.0188,  0.0038,  ...,  0.1389,  0.1539,  0.0939]],
 
         [[-0.3000, -0.2857, -0.3284,  ..., -0.2146,  0.6244,  1.2216],
          [-0.3426, -0.3000,

In [ ]:
from torch import Tensor

input_ids = Tensor(
    [
        [
            1,
            319,
            13563,
            1546,
            263,
            12758,
            1404,
            322,
            385,
            23116,
            21082,
            20255,
            29889,
            450,
            20255,
            4076,
            8444,
            29892,
            13173,
            29892,
            322,
            1248,
            568,
            6089,
            304,
            278,
            1404,
            29915,
            29879,
            5155,
            29889,
            3148,
            1001,
            29901,
            29871,
            -200,
            29871,
            13,
            3624,
            278,
            1081,
            2527,
            304,
            278,
            2175,
            470,
            304,
            278,
            1492,
            310,
            278,
            2022,
            393,
            278,
            10992,
            23090,
            338,
            304,
            278,
            1492,
            310,
            29973,
            13,
            22550,
            278,
            1139,
            773,
            263,
            2323,
            1734,
            470,
            16549,
            29889,
            319,
            1799,
            9047,
            13566,
            29901,
        ]
    ],
)
attention_mask = Tensor([
    [
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
        True,
    ]
])


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5", trust_remote_code=True)


In [14]:
import torch

# 创建符合要求尺寸的零张量 [batch_size, channels, height, width]
# CLIP模型通常使用3通道RGB图像
dummy_image = torch.zeros(1, 3, 336, 336).to("cuda").to(torch.bfloat16)
input_ids = input_ids.to("cuda").to(torch.int32)
attention_mask = attention_mask.to("cuda").to(torch.bfloat16)
model = model.to("cuda")
output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    images=dummy_image,
    pad_token_id=0,
    # do_sample=False,
    # temperature=0.0,
    # top_p=1.0,
    num_beams=1,
    # max_new_tokens=16,
    use_cache=True,
)


None
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1.

In [15]:
output


tensor([[   13, 29909,     2]], device='cuda:0')

In [ ]:
tokenizer.batch_decode(output, skip_special_tokens=True)


['\nA']

In [19]:
tokenizer.batch_decode(input_ids, skip_special_tokens=True)


TypeError: argument 'ids': 'float' object cannot be interpreted as an integer